In [21]:
from IPython.display import display, clear_output, Markdown
import ipywidgets as widgets
from widgets.upload import get_upload_widget
from tagging.sugest import sugerir_rotulos
from extract import extract_data
import io

from database import FinanceDB
import pandas as pd

In [ ]:

upload_widget = get_upload_widget()


def process_row(df, index, labels):
    row = df.iloc[index]
    clear_output(wait=True)
    
    text = f"### Item {index}\n"
    for k, v in row.items():
        text += f"**{k}**: {v}  \n"  # Use dois espaços antes do \n para quebra no Markdown
    display(Markdown(text))

    


    input_text = widgets.Text(description="Label:", placeholder="Digite o rótulo aqui")
    nome_text = widgets.Text(description="Nome:", placeholder="Digite o nome aqui")

    if rotulos := sugerir_rotulos(row["Data"], row["Valor"], row["Destino / Origem"], row["Descricao"]):
        input_text.value = rotulos["Subcategoria"]
        nome_text.value = rotulos["Nome"]

    button = widgets.Button(description="Confirmar")
    
    output = widgets.Output()
    
    def on_confirm(b):
        global upload_widget
        nonlocal index
        label = input_text.value.strip()
        if not label:
            with output:
                clear_output()
                print("Por favor, insira um rótulo antes de confirmar.")
            return
        
        labels.append(label)
        
        if index + 1 < len(df):
            process_row(df, index + 1, labels)
        else:
            clear_output()
            print("Fim da lista. Labels cadastrados:")
            for i, lab in enumerate(labels):
                print(f"Row {i}: {lab}")
            
            upload_widget = get_upload_widget()
            upload_widget.observe(handle_upload, names="value")
            display(upload_widget)
    
    button.on_click(on_confirm)
    display(input_text, nome_text, button, output)
    


def handle_upload(_):
    if value:=upload_widget.value:
        file_info = next(iter(value.values()))
        content = file_info["content"]
        df = extract_data(io.BytesIO(content))
        labels = []
        process_row(df, 0, labels)

    
    
upload_widget.observe(handle_upload, names="value")
display(upload_widget)

### Item 2
**Data**: 2024-05-02 07:07:52  
**Valor**: -500.0  
**Destino / Origem**: para conta investimento  
**Descricao**: Transferência enviada para conta investimento  
**Tipo**: Transferência  


Text(value='Aporte para Investimento', description='Label:', placeholder='Digite o rótulo aqui')

Text(value='Pedro Paulo Carvalho Vieira (si mesmo, conta investimento)', description='Nome:', placeholder='Dig…

Button(description='Confirmar', style=ButtonStyle())

Output()